In [39]:
import os
import json

import pandas as pd


directory = "treesatai"
with open(os.path.join(directory, "treesatai-results.json")) as f:
    results = json.load(f)

df = pd.DataFrame.from_dict(results).transpose()
df["rgb"] = ["RGB" if "rgb" in model_name else "MSI" for model_name in df.index]
df["size"] = [int(model_name.split("_")[-1]) for model_name in df.index]
df["encoder"] = [
    model_name.rsplit("_", 1)[0].replace("_rgb", "") for model_name in df.index
]
df = df.sort_values(["rgb", "encoder", "size"], ascending=True)
df.to_csv("treesatai-results.csv")
df

,mAP_weighted,mAP_micro,f1_weighted,f1_micro,precision_micro,precision_weighted,recall_micro,recall_weighted,accuracy,rgb,size,encoder
imagestats_34,0.286503,0.270979,0.359495,0.349524,0.349524,0.393434,0.349524,0.349524,0.349524,MSI,no,imagestats
imagestats_224,0.286503,0.270979,0.359495,0.349524,0.349524,0.393434,0.349524,0.349524,0.349524,MSI,yes,imagestats
mosaiks_512_3_34,0.286215,0.270447,0.355273,0.345163,0.345163,0.388608,0.345163,0.345163,0.345163,MSI,no,mosaiks_512_3
mosaiks_512_3_224,0.283875,0.268987,0.351697,0.342982,0.342982,0.382046,0.342982,0.342982,0.342982,MSI,yes,mosaiks_512_3
resnet18_pretrained_moco_34,0.261722,0.245449,0.337014,0.328311,0.328311,0.380030,0.328311,0.328311,0.328311,MSI,no,resnet18_pretrained_moco
resnet18_pretrained_moco_224,0.280094,0.264168,0.349149,0.337034,0.337034,0.395083,0.337034,0.337034,0.337034,MSI,yes,resnet18_pretrained_moco
resnet50_pretrained_imagenet_34,0.179225,0.153408,0.225946,0.219469,0.219469,0.255205,0.219469,0.219469,0.219469,MSI,no,resnet50_pretrained_imagenet
resnet50_pretrained_imagenet_224,0.226174,0.204940,0.287149,0.283703,0.283703,0.314323,0.283703,0.283703,0.283703,MSI,yes,resnet50_pretrained_imagenet
resnet50_pretrained_moco_34,0.244184,0.224896,0.308125,0.298573,0.298573,0.350240,0.298573,0.298573,0.298573,MSI,no,resnet50_pretrained_moco
resnet50_pretrained_moco_224,0.322002,0.308999,0.390872,0.378866,0.378866,0.428131,0.378866,0.378866,0.378866,MSI,yes,resnet50_pretrained_moco
